In [1]:
# Make sure you have a GPU running
!nvidia-smi

Wed Jul 12 04:37:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Install the latest release of Haystack in your own environment
! pip install farm-haystack

# Install the latest master of Haystack
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

DEPRECATION: git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-vt4tkzlv/farm-haystack_c5748b98ec9d4d919184a80db049b8ed
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-vt4tkzlv/farm-haystack_c5748b98ec9d4d919184a80db049b8ed
  Resolved https://github.com/deepset-ai/haystack.git to commit b5aef24a7ebac55cb4ba492baf81a85598700b94
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
!pip install sentence-transformers

In [4]:
from haystack.nodes import FARMReader

In [5]:
from haystack.nodes import FARMReader
reader = FARMReader(model_name_or_path="distilbert-base-uncased-distilled-squad", use_gpu=True)

In [6]:
data_dir = "data/"

In [7]:
reader.train(data_dir=data_dir, train_filename="dev-v2.0.json", use_gpu=True, n_epochs=1, save_dir="my_model",)

Train epoch 0/0 (Cur. train loss: 1.1962): 100%|██████████| 1360/1360 [05:12<00:00,  4.35it/s]


In [8]:
# If you want to load it at a later point, just do:
new_reader = FARMReader(model_name_or_path="my_model")

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [9]:
reader_eval_results = new_reader.eval_on_file("data/", "dev-v2.0.json", device="cuda")

- instead of giving you full control over which labels to use, this method always returns three types of metrics: combined (no suffix), text_answer ('_text_answer' suffix) and no_answer ('_no_answer' suffix) metrics.
- instead of comparing predictions with labels on a string level, this method compares them on a token-ID level. This makes it unable to do any string normalization (e.g. normalize whitespaces) beforehand.
Hence, results might slightly differ from those of `Pipeline.eval()`
.If you are just about starting to evaluate your model consider using `Pipeline.eval()` instead.
Evaluating: 100%|██████████| 272/272 [01:44<00:00,  2.59it/s]


In [10]:
reader_eval_results

{'EM': 52.114882506527415,
 'f1': 52.138505532761414,
 'top_n_accuracy': 99.32114882506528,
 'top_n': 4,
 'EM_text_answer': 6.784861916126833,
 'f1_text_answer': 6.831133408017146,
 'top_n_accuracy_text_answer': 98.67030344357313,
 'top_n_EM_text_answer': 92.77190589839755,
 'top_n_f1_text_answer': 96.81932415015089,
 'Total_text_answer': 2933,
 'EM_no_answer': 99.3954480796586,
 'f1_no_answer': 99.3954480796586,
 'top_n_accuracy_no_answer': 100.0,
 'Total_no_answer': 2812}

In [11]:
context = '''Chennai Super Kings (CSK) is a professional cricket franchise based in Chennai, Tamil Nadu, India, that competes in the Indian Premier League. Founded in 2008, the team plays its home matches at the M. A. Chidambaram Stadium in Chennai. The team is owned by India Cements through its Chennai Super Kings Cricket Limited holding company. They have won a record five IPL titles (shared with Mumbai Indians), appeared in a record 10 finals and have qualified for the playoff stages 12 times out of the 14 seasons they have played, more than any other team.[1] The team has been captained by MS Dhoni since inception and is currently coached by Stephen Fleming. In January 2022, CSK became India's first unicorn sports enterprise.[2]

The team was suspended for two years from the IPL starting July 2015 due to the involvement of its owners in the 2013 IPL betting case,[3] and won the title in its comeback season of 2018. In addition, they have also won the Champions League Twenty20 in 2010 and 2014. They are amongst the most valuable IPL franchises with a valuation of roughly $1.15 billion as of 2022.[4]'''

In [12]:
new_reader.predict_on_texts("who is the captain of CSK",[context])

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 47.26 Batches/s]


{'query': 'who is the captain of CSK',
 'no_ans_gap': 0.5551280975341797,
 'answers': [<Answer {'answer': 'MS Dhoni', 'type': 'extractive', 'score': 0.5396232008934021, 'context': 've played, more than any other team.[1] The team has been captained by MS Dhoni since inception and is currently coached by Stephen Fleming. In Januar', 'offsets_in_document': [{'start': 588, 'end': 596}], 'offsets_in_context': [{'start': 71, 'end': 79}], 'document_ids': ['2ecdc2082153aae666a5cc9de5abb27d'], 'meta': {}}>]}

In [14]:
from haystack import Pipeline, Document
from haystack.utils import print_answers

p = Pipeline()
p.add_node(component=new_reader, name="Reader", inputs=["Query"])
res = p.run(
    query="who is the captain of CSK", documents=[Document(content=context)]
)
print_answers(res,details="medium")

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 29.30 Batches/s]

'Query: who is the captain of CSK'
'Answers:'
[   {   'answer': 'MS Dhoni',
        'context': 've played, more than any other team.[1] The team has been '
                   'captained by MS Dhoni since inception and is currently '
                   'coached by Stephen Fleming. In Januar',
        'score': 0.5396232008934021}]
